In [54]:
# Prospects list

from datetime import datetime

current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

co_list = []
company_dict = {
        "company_name": "Basis",
        "domain": "getbasis.com",
        "linkedin_url": "linkedin.com/company/basis/",
        "created_at": current_datetime
        }
co_list.append(company_dict)

company_dict_2 = {
        "company_name": "Google",
        "domain": "google.com",
        "linkedin_url": "linkedin.com/company/google/",
        "created_at": current_datetime
        }

co_list.append(company_dict_2)

In [55]:
print(co_list)

[{'company_name': 'Basis', 'domain': 'getbasis.com', 'linkedin_url': 'linkedin.com/company/basis/', 'created_at': '2022-03-23 21:09:38'}, {'company_name': 'Google', 'domain': 'google.com', 'linkedin_url': 'linkedin.com/company/google/', 'created_at': '2022-03-23 21:09:38'}]


In [61]:
# People Data Labs Company Enrichment API

import requests

https://api.peopledatalabs.com/v5/company/search

url = "https://api.peopledatalabs.com/v5/company/enrich"

querystring = {"website":"www.facebook.com"}

headers = {
    'accept': "application/json",
    'content-type': "application/json",
    'x-api-key': "XXXX"
    }

responses = []

for item in co_list:
    querystring = {"website":item["domain"]}
    response = requests.request("GET", url, headers=headers, params=querystring)
    responses.append({item["company_name"]:response.text})

print(responses)

[{'Basis': '{"status": 400, "error": {"type": ["invalid_request_error"], "message": "No query specified. Use either the `query` or `sql` parameters for elasticsearch queries or SQL queries, respectively."}}'}, {'Google': '{"status": 400, "error": {"type": ["invalid_request_error"], "message": "No query specified. Use either the `query` or `sql` parameters for elasticsearch queries or SQL queries, respectively."}}'}]


In [62]:
# Using Company Search from People Data Labs (currently in Beta)

import requests, json

API_KEY = "XXXX"

PDL_URL = "https://api.peopledatalabs.com/v5/company/search"

H = {
  'Content-Type': "application/json",
  'X-api-key': API_KEY
}

SQL_QUERY = \
"""
  SELECT * FROM company
  WHERE website='google.com';
 """

P = {
  'sql': SQL_QUERY,
  'size': 10,
  'pretty': True
}

response = requests.get(
  PDL_URL,
  headers=H,
  params=P
).json()

if response["status"] == 200:
  data = response['data']
  with open("my_pdl_search.jsonl", "w") as out:
    for record in data:
      out.write(json.dumps(record) + "\n")
  print(f"successfully grabbed {len(data)} records from pdl")
  print(f"{response['total']} total pdl records exist matching this query")
else:
  print("NOTE. The carrier pigeons lost motivation in flight. See error and try again.")
  print("error:", response)

successfully grabbed 1 records from pdl
1 total pdl records exist matching this query


In [64]:
data

[{'name': 'google',
  'size': '10001+',
  'employee_count': 272189,
  'id': 'google',
  'founded': 1998,
  'industry': 'internet',
  'location': {'name': 'mountain view, california, united states',
   'locality': 'mountain view',
   'region': 'california',
   'metro': 'san francisco, california',
   'country': 'united states',
   'continent': 'north america',
   'street_address': '1600 amphitheatre parkway',
   'address_line_2': None,
   'postal_code': '94043',
   'geo': None},
  'linkedin_id': '1441',
  'linkedin_url': 'linkedin.com/company/google',
  'facebook_url': 'facebook.com/google',
  'twitter_url': None,
  'profiles': ['linkedin.com/company/google',
   'linkedin.com/company/1441',
   'facebook.com/google',
   'crunchbase.com/organization/google'],
  'website': 'google.com',
  'ticker': 'GOOGL',
  'type': 'public',
  'summary': 'google’s mission is to organize the world‘s information and make it universally accessible and useful.\n\n since our founding in 1998, google has grown

In [66]:
# Next step :
# Scoring

# Total Company Lead Score Column Value (starting at 0) should be updated based on the following parameters (provided from the enrichment of company data from PDL):

# If value exists, add the associated “point” total to the Total Lead Score Column Value.  If any other value is in the table, do not update total Value. 

# size
# 0-10 +10
# 11-50 +5
# 51-200 +3
# 201-500 +3
# 501-1000 -5
# 1001-5000 -10
# Founded
# 2022 +10
# 2021 +10
# 2020 +5
# 2015-2019 +3
# 2005-2014 +2
# Industry
# computer software +10
# design +5
# e-learning +5
# information services +5
# information technology and services +10
# internet +10
# program development +5
# Linkedin_id
# value present +5
# null -5
# Location Country
# United States +10
# australia +8
# canada +8
# united kingdom +8
# type
# Private +5
# Tags
# data +5
# Saas +10
# api +10
# software +10
